<table align="center">
  <td align="center"><a target="_blank" href="http://introtodeeplearning.com">
        <img src="https://i.ibb.co/Jr88sn2/mit.png" style="padding-bottom:5px;" />
      Visit MIT Deep Learning</a></td>
  <td align="center"><a target="_blank" href="https://colab.research.google.com/github/MITDeepLearning/introtodeeplearning/blob/master/lab3/LLM_Finetuning.ipynb">
        <img src="https://i.ibb.co/2P3SLwK/colab.png"  style="padding-bottom:5px;" />Run in Google Colab</a></td>
  <td align="center"><a target="_blank" href="https://github.com/MITDeepLearning/introtodeeplearning/blob/master/lab3/LLM_Finetuning.ipynb">
        <img src="https://i.ibb.co/xfJbPmL/github.png"  height="70px" style="padding-bottom:5px;"  />View Source on GitHub</a></td>
</table>

# Copyright Information

In [ ]:
# Copyright 2025 MIT Introduction to Deep Learning. All Rights Reserved.
#
# Licensed under the MIT License. You may not use this file except in compliance
# with the License. Use and/or modification of this code outside of MIT Introduction
# to Deep Learning must reference:
#
# © MIT Introduction to Deep Learning
# http://introtodeeplearning.com
#

In [ ]:
# Install and import MIT Deep Learning utilities
!pip install mitdeeplearning > /dev/null 2>&1
import mitdeeplearning as mdl

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
import numpy as np

import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader

from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
from lion_pytorch import Lion

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
# Basic question-answer template
template_without_answer = "<start_of_turn>user\n{question}<end_of_turn>\n<start_of_turn>model\n"
template_with_answer = template_without_answer + "{answer}<end_of_turn>\n"

# Let's try to put something into the template to see how it looks
print(template_with_answer.format(question="What is your name?", answer="My name is Gemma!"))

<start_of_turn>user
What is your name?<end_of_turn>
<start_of_turn>model
My name is Gemma!<end_of_turn>



In [ ]:
# Load the tokenizer for Gemma 2B
model_id = "unsloth/gemma-2-2b-it"
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Add a padding token if it doesn't exist
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# How big is the tokenizer?
print(f"Vocab size: {len(tokenizer.get_vocab())}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type SwigPyPacked has no __module__ attribute
<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type SwigPyObject has no __module__ attribute


Vocab size: 256000


In [ ]:
# Lets test out both steps:
text = "Here is some sample text!"
print(f"Original text: {text}")

# Tokenize the text
tokens = tokenizer.encode(text, return_tensors="pt")
print(f"Encoded tokens: {tokens}")

# Decode the tokens
decoded_text = tokenizer.decode(tokens[0], skip_special_tokens=True)
print(f"Decoded text: {decoded_text}")

Original text: Here is some sample text!
Encoded tokens: tensor([[     2,   4858,    603,   1009,   6453,   2793, 235341]])
Decoded text: Here is some sample text!


This is really cool. Now we have a way to move in and out of the token space.

To "chat" with our LLM chatbot, we need to use the tokenizer and the chat template together, in order for the model to respond to the user's question. We can use the templates defined earlier to construct a prompt for the model, without the answer.

In [ ]:
prompt = template_without_answer.format(question="What is the capital of France? Use one word.")
print(prompt)

<start_of_turn>user
What is the capital of France? Use one word.<end_of_turn>
<start_of_turn>model



If we were to feed this to the model, it would see that it is now the start of the model's turn, and it would generate the answer to this question.

In [ ]:
# Load the model -- note that this may take a few minutes
# Load the model -- note that this may take a few minutes
def apply_lora(model):
    # Define LoRA config
    lora_config = LoraConfig(
        r=8, # rank of the LoRA matrices
        task_type="CAUSAL_LM",
        target_modules=[
            "q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"
        ],
    )

    # Apply LoRA to the model
    lora_model = get_peft_model(model, lora_config)
    return lora_model

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    dtype=torch.float16,
    low_cpu_mem_usage=True,
)
# Optional: enable gradient checkpointing to save memory
if hasattr(model, "gradient_checkpointing_enable"):
    model.gradient_checkpointing_enable()
# Resize embeddings (pad token added) and attach LoRA adapters
model.resize_token_embeddings(len(tokenizer))
model = apply_lora(model)

if hasattr(model, "print_trainable_parameters"):
    model.print_trainable_parameters()
else:
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total = sum(p.numel() for p in model.parameters())
    print(f"Trainable params: {trainable} / {total} ({trainable/total*100:.2f}%)")


trainable params: 10,383,360 || all params: 2,624,725,248 || trainable%: 0.3956


In [ ]:
### Putting it together to prompt the model and generate a response ###

# 1. Construct the prompt in chat template form
question = "What does MIT stand for?"
prompt = template_without_answer.format(question=question)

# 2. Tokenize the prompt, including attention mask
inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)

# 3. Generate a sequence of tokens for the answer
with torch.no_grad():
    gen_ids = model.generate(
        inputs["input_ids"],
        attention_mask=inputs["attention_mask"],  # Add attention mask
        max_new_tokens=50,  # Increased tokens
        do_sample=True,  # Enable sampling
        temperature=0.7 # Added temperature
    )

# 4. Decode and print the full text
print(tokenizer.decode(gen_ids[0], skip_special_tokens=True))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


user
What does MIT stand for?
model
MIT stands for **Massachusetts Institute of Technology**. 



In [ ]:
prompt = template_without_answer.format(question="What does MIT stand for?")
tokens = tokenizer.encode(prompt, return_tensors="pt").to(model.device)
output = model.generate(tokens, max_new_tokens=20)
print(tokenizer.decode(output[0]))

<bos><start_of_turn>user
What does MIT stand for?<end_of_turn>
<start_of_turn>model
MIT stands for **Massachusetts Institute of Technology**. 
<end_of_turn>


In [ ]:
# LoRA is a way to finetune LLMs very efficiently by only updating a small subset of the model's parameters



### 1.3.3: Forward pass and loss computation

Now let's define a function to perform a forward pass through the LLM and compute the loss. The forward pass gives us the logits -- which reflect the probability distribution over the next token -- for the next token. We can compute the loss by comparing the predicted logits to the true next token -- our target label. Note that this is effectively a classification problem! So, our loss can be captured by the cross entropy loss, and we can use PyTorch's [`nn.functional.cross_entropy`](https://pytorch.org/docs/stable/generated/torch.nn.functional.cross_entropy.html) function to compute it.

In [ ]:
def forward_and_compute_loss(model, tokens, mask, context_length=512):
    # Truncate to context length
    tokens = tokens[:, :context_length]
    mask = mask[:, :context_length]

    # Construct the input, output, and mask
    x = tokens[:, :-1]
    y = tokens[:, 1:]
    mask = mask[:, 1:]

    # Forward pass to compute logits
    logits = model(x).logits

    # Compute loss
    loss = F.cross_entropy(
        logits.view(-1, logits.size(-1)),
        y.view(-1),
        reduction="none"
    )

    # Mask out the loss for non-answer tokens
    loss = loss[mask.view(-1)].mean()

    return loss

## PII Masking Fine-tuning (source_text ➜ target_text)

Adapt the pipeline to a PII masking dataset with columns: `source_text`, `target_text`, `privacy_mask`, `span_labels`, `mbert_text_tokens`, `mbert_bio_labels`, `id`, `language`, `set`. We will fine-tune the model to transform `source_text` into its masked form `target_text`.


In [ ]:
# Prompt template specialized for PII masking with few-shot examples

def build_pii_prompt(source_text: str) -> str:
    instruction = (
        "You are a data privacy assistant. Mask all personally identifiable information (PII) "
        "in the following text using the masking scheme shown in the examples. "
        "Output only the masked text.\n\n"
        "Examples:\n\n"
        "Input: My name is John Smith and my email is john.smith@email.com\n"
        "Output: My name is [FIRSTNAME] [LASTNAME] and my email is [EMAIL]\n\n"
        "Input: Call me at 555-123-4567 or visit 123 Main Street, Boston MA 02101\n"
        "Output: Call me at [PHONENUMBER] or visit [STREET] [CITY] [STATE] [ZIPCODE]\n\n"
        "Input: My username is alice_2023 and I was born on 03/15/1990\n"
        "Output: My username is [USERNAME] and I was born on [DOB]\n\n"
        "Input: The SSN is 123-45-6789 and account number is ACC98765\n"
        "Output: The SSN is [SSN] and account number is [ACCOUNTNUMBER]"
    )
    return (
        f"<start_of_turn>user\n{instruction}\n\nText:\n{source_text}\n<end_of_turn>\n"
        f"<start_of_turn>model\n"
    )


In [ ]:
# Load a PII masking dataset (expects the listed columns)
# If you have a local file instead of HF dataset, replace with pandas read_csv and Dataset.from_pandas
from datasets import load_dataset

try:
    pii_ds = load_dataset("ai4privacy/pii-masking-300k")
except Exception as e:
    print("Falling back: please provide a local dataset with required columns.")
    raise e

# Keep English and split by provided 'set' column if present
if "language" in pii_ds["train"].column_names:
    pii_ds = pii_ds.filter(lambda ex: ex.get("language", "en") == "English")

# Train/validation split: if dataset has 'set' use it, else do a random split
if "set" in pii_ds["train"].column_names:
    train_split = pii_ds["train"].filter(lambda ex: ex.get("set", "train") == "train")
    valid_split = pii_ds["validation"].filter(lambda ex: ex.get("set", "validation") == "validation")
else:
    split = pii_ds["train"].train_test_split(test_size=0.02, seed=42)
    train_split, valid_split = split["train"], split["test"]

needed_columns = [
    "source_text", "target_text", "privacy_mask", "span_labels",
    "mbert_text_tokens", "mbert_bio_labels", "id", "language", "set"
]

missing = [c for c in ["source_text", "target_text"] if c not in train_split.column_names]
assert not missing, f"Dataset missing required columns: {missing}"

# Map to prompt-target fields the model will use

def add_prompt_fields(example):
    src = example["source_text"]
    tgt = example["target_text"]
    prompt = build_pii_prompt(src)
    example["prompt"] = prompt
    example["target"] = tgt
    return example

# Apply mapping and remove original columns not in needed_columns, keeping the new 'prompt' and 'target'
original_columns = train_split.column_names
columns_to_remove = [col for col in original_columns if col not in needed_columns]

train_split = train_split.map(add_prompt_fields, remove_columns=columns_to_remove)
valid_split = valid_split.map(add_prompt_fields, remove_columns=columns_to_remove)


print("PII dataset:", train_split)
print("Columns:", train_split.column_names[:10])
print({k: train_split[k][0] for k in ["prompt", "target"]})
print("Valid Split -> ", len(valid_split))

Filter:   0%|          | 0/7946 [00:00<?, ? examples/s]

Map:   0%|          | 0/29908 [00:00<?, ? examples/s]

Map:   0%|          | 0/7946 [00:00<?, ? examples/s]

PII dataset: Dataset({
    features: ['source_text', 'target_text', 'privacy_mask', 'span_labels', 'mbert_text_tokens', 'mbert_bio_labels', 'id', 'language', 'set', 'prompt', 'target'],
    num_rows: 29908
})
Columns: ['source_text', 'target_text', 'privacy_mask', 'span_labels', 'mbert_text_tokens', 'mbert_bio_labels', 'id', 'language', 'set', 'prompt']
{'prompt': '<start_of_turn>user\nYou are a data privacy assistant. Mask all personally identifiable information (PII) in the following text using the masking scheme shown in the examples. Output only the masked text.\n\nText:\nSubject: Group Messaging for Admissions Process\n\nGood morning, everyone,\n\nI hope this message finds you well. As we continue our admissions processes, I would like to update you on the latest developments and key information. Please find below the timeline for our upcoming meetings:\n\n- wynqvrh053 - Meeting at 10:20am\n- luka.burg - Meeting at 21\n- qahil.wittauer - Meeting at quarter past 13\n- gholamhossein

In [ ]:
# Collate to create input_ids and labels where prompt tokens are ignored (-100)
from torch.utils.data import DataLoader

def tokenize_with_labels(batch, tokenizer, max_length=512):
    prompts = batch["prompt"]
    targets = batch["target"]

    # Tokenize separately so we can compute boundaries
    prompt_enc = tokenizer(prompts, padding=False, truncation=True, max_length=max_length)
    target_enc = tokenizer(targets, padding=False, truncation=True, max_length=max_length)

    input_ids = []
    labels = []
    attention_mask = []

    for p_ids, t_ids in zip(prompt_enc["input_ids"], target_enc["input_ids"]):
        # Build concatenated sequence: [prompt] + [target]
        ids = p_ids + t_ids
        ids = ids[:max_length]

        # Build labels: -100 for prompt tokens, target token ids for target span
        prompt_len = min(len(p_ids), max_length)
        tgt_len = max(0, min(len(t_ids), max_length - prompt_len))
        lab = ([-100] * prompt_len) + t_ids[:tgt_len]
        lab = lab[:len(ids)]

        am = [1] * len(ids)

        input_ids.append(ids)
        labels.append(lab)
        attention_mask.append(am)

    # Pad to max batch length
    batch_max = max(len(x) for x in input_ids)
    def pad_to(x, pad_id):
        return x + [pad_id] * (batch_max - len(x))

    pad_id = tokenizer.pad_token_id or tokenizer.eos_token_id
    input_ids = [pad_to(x, pad_id) for x in input_ids]
    labels = [pad_to(x, -100) for x in labels]
    attention_mask = [pad_to(x, 0) for x in attention_mask]

    # Create tensors on CPU first, then move to device
    return {
        "input_ids": torch.tensor(input_ids, dtype=torch.long).to(model.device),
        "labels": torch.tensor(labels, dtype=torch.long).to(model.device),
        "attention_mask": torch.tensor(attention_mask, dtype=torch.long).to(model.device),
    }

# Ensure pad token is set
if tokenizer.pad_token_id is None and tokenizer.eos_token_id is not None:
    tokenizer.pad_token = tokenizer.eos_token

# Use the reduced BATCH_SIZE
BATCH_SIZE = 8 # Reduced batch size
MAX_LEN = 512

train_loader_pii = DataLoader(
    train_split, batch_size=BATCH_SIZE, shuffle=True,
    collate_fn=lambda b: tokenize_with_labels({k: [ex[k] for ex in b] for k in b[0]}, tokenizer, max_length=MAX_LEN)
)

valid_loader_pii = DataLoader(
    valid_split, batch_size=BATCH_SIZE, shuffle=False,
    collate_fn=lambda b: tokenize_with_labels({k: [ex[k] for ex in b] for k in b[0]}, tokenizer, max_length=MAX_LEN)
)

print("Train batches (PII):", len(train_loader_pii))

Train batches (PII): 29908


In [ ]:
# Training loop for PII masking (text-to-text)
from lion_pytorch import Lion

def train_pii(model, dataloader, val_loader=None, max_steps=200, learning_rate=1e-4):
    model.train()
    optimizer = Lion(model.parameters(), lr=learning_rate)
    losses = []

    step = 0
    for batch in dataloader:
        optimizer.zero_grad()
        outputs = model(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            labels=batch["labels"],
        )
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        losses.append(loss.item())
        if step % 10 == 0:
            print(f"step {step} loss: {loss.item():.4f}")

        step += 1
        if step >= max_steps:
            break

    return model

# Kick off a short fine-tune (adjust steps as needed)
# model = train_pii(model, train_loader_pii, valid_loader_pii, max_steps=200, learning_rate=1e-4)


In [ ]:
# Inference helper for PII masking

def pii_mask(text: str, max_new_tokens=128, temperature=0.0, only_answer=True):
    prompt = build_pii_prompt(text)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        gen = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=(temperature > 0.0),
            temperature=max(temperature, 1e-6),
        )

    out = tokenizer.decode(gen[0], skip_special_tokens=True)
    if only_answer:
        # Return only the model turn after the prompt
        if "<start_of_turn>model" in out:
            out = out.split("<start_of_turn>model")[-1]
        return out.strip()
    return out

# Example:
# print(pii_mask("My name is John Smith and my SSN is 123-45-6789. I live at 10 Main St, Boston MA."))


In [ ]:
# PII evaluation: exact match (EM) and simple character-level F1
import numpy as np
from collections import Counter

@torch.no_grad()
def evaluate_pii_exact_match(dset, n=200):
    model.eval()
    n = min(n, len(dset))
    ems = []
    for i in range(n):
        src = dset[i]["source_text"]
        tgt = dset[i]["target_text"]
        pred = pii_mask(src, max_new_tokens=256, temperature=0.0, only_answer=True)
        ems.append(1.0 if pred.strip() == tgt.strip() else 0.0)
    em = float(np.mean(ems)) if ems else 0.0
    print(f"Exact Match (EM) over {n} samples: {em:.3f}")
    return em

def char_f1(pred: str, gold: str):
    pc = Counter(pred)
    gc = Counter(gold)
    overlap = sum((pc & gc).values())
    if overlap == 0:
        return 0.0
    precision = overlap / max(1, sum(pc.values()))
    recall = overlap / max(1, sum(gc.values()))
    if precision + recall == 0:
        return 0.0
    return 2 * precision * recall / (precision + recall)

@torch.no_grad()
def evaluate_pii_char_f1(dset, n=200):
    model.eval()
    n = min(n, len(dset))
    scores = []
    for i in range(n):
        src = dset[i]["source_text"]
        tgt = dset[i]["target_text"]
        pred = pii_mask(src, max_new_tokens=256, temperature=0.0, only_answer=True)
        scores.append(char_f1(pred, tgt))
    f1 = float(np.mean(scores)) if scores else 0.0
    print(f"Char-level F1 over {n} samples: {f1:.3f}")
    return f1

# Example:
# _ = evaluate_pii_exact_match(valid_split, n=100)
# _ = evaluate_pii_char_f1(valid_split, n=100)


In [ ]:
FT_STEPS = 2000
LR = 1e-4

model.train()
model = train_pii(model, train_loader_pii, valid_loader_pii, max_steps=FT_STEPS, learning_rate=LR)

print("Fine-tuning completed.")

step 0 loss: 0.0874
step 10 loss: 0.1280
step 20 loss: 0.0633
step 30 loss: 0.0749
step 40 loss: 0.4524
step 50 loss: 0.0464
step 60 loss: 0.1240
step 70 loss: 0.0729
step 80 loss: 0.0446
step 90 loss: 0.0762
step 100 loss: 0.0051
step 110 loss: 0.1852
step 120 loss: 0.0211
step 130 loss: 0.0585
step 140 loss: 0.1228
step 150 loss: 0.0466
step 160 loss: 0.3000
step 170 loss: 0.0100
step 180 loss: 0.0721
step 190 loss: 0.0003
Fine-tuning completed.


In [ ]:
N_SHOW = 3
for i in range(N_SHOW):
    src = valid_split[i]["source_text"]
    tgt = valid_split[i]["target_text"]
    pred = pii_mask(src, max_new_tokens=256, temperature=0.0, only_answer=True)
    print(f"--- Sample {i} ---")
    print("Source:\n", src[:500])
    print("\nTarget:\n", tgt[:500])
    print("\nPrediction:\n", pred[:500])
    print()

In [ ]:
# === Evaluation: fine-tuned vs base model (EM and char-F1) ===
from transformers import AutoModelForCausalLM

@torch.no_grad()
def pii_mask_with(model_obj, text: str, max_new_tokens=256):
    prompt = build_pii_prompt(text)
    inputs = tokenizer(prompt, return_tensors="pt").to(model_obj.device)
    gen = model_obj.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,
    )
    out = tokenizer.decode(gen[0], skip_special_tokens=True)
    if "<start_of_turn>model" in out:
        out = out.split("<start_of_turn>model")[-1]
    return out.strip()

# Load a fresh base model (no LoRA) for comparison
base_model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")

N_EVAL = 100  # Increased from 1 to 100 for better statistical significance
ems_ft, ems_base = [], []
f1s_ft, f1s_base = [], []

print(f"Evaluating on {N_EVAL} samples...\n")

for i in range(min(N_EVAL, len(valid_split))):
    src = valid_split[i]["source_text"]
    tgt = valid_split[i]["target_text"]

    # Fine-tuned prediction (using global model)
    pred_ft = pii_mask(src, max_new_tokens=256, temperature=0.0, only_answer=True)

    # Base prediction
    pred_base = pii_mask_with(base_model, src, max_new_tokens=256)

    ems_ft.append(1.0 if pred_ft.strip() == tgt.strip() else 0.0)
    ems_base.append(1.0 if pred_base.strip() == tgt.strip() else 0.0)

    f1s_ft.append(char_f1(pred_ft, tgt))
    f1s_base.append(char_f1(pred_base, tgt))
    
    # Print progress every 20 samples
    if (i + 1) % 20 == 0:
        print(f"Evaluated {i + 1}/{N_EVAL} samples...")

print("\n" + "="*60)
print("FINAL RESULTS")
print("="*60)
print(f"Fine-tuned EM: {np.mean(ems_ft):.3f} | Base EM: {np.mean(ems_base):.3f}")
print(f"Fine-tuned F1: {np.mean(f1s_ft):.3f} | Base F1: {np.mean(f1s_base):.3f}")
print(f"\nImprovement: {(np.mean(f1s_ft) - np.mean(f1s_base)) / np.mean(f1s_base) * 100:.1f}%")
print("="*60)

Fine-tuned EM: 0.000 | Base EM: 0.000
Fine-tuned F1: 0.268 | Base F1: 0.526
